In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [26]:
import gzip, pickle, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from fastai import datasets
import torch

In [27]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [28]:
x_train,y_train,x_valid,y_valid = get_data()

In [29]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [30]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [31]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(3.0614e-05), tensor(1.))

In [32]:
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

In [33]:
nh = 50

# Architecture

In [35]:
import math
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [36]:
x_valid.mean(), x_valid.std()

(tensor(-0.0058), tensor(0.9924))

In [37]:
def lin(x, w, b): return x@w + b

In [38]:
t = lin(x_valid, w1, b1)

In [39]:
t.mean(), t.std()

(tensor(-0.0260), tensor(1.0189))

In [40]:
def relu(x): return x.clamp_min(0.)

In [41]:
# Forward pass l2(relu(l1))
t = relu(lin(x_valid, w1, b1))

In [42]:
t.mean(), t.std()

(tensor(0.3907), tensor(0.5864))

In [43]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [44]:
w1.mean(), w2.std()

(tensor(-4.6631e-05), tensor(0.1475))

In [45]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.4944), tensor(0.7925))

In [46]:
from torch.nn import init

In [47]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # lin
t = relu(lin(x_valid, w1, b1))

In [48]:
t.mean(), t.std()

(tensor(0.4910), tensor(0.7645))

In [49]:
init.kaiming_normal_??

Signature: init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where

    .. math::
        \text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan\_in}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``'fan_

In [50]:
w1.mean(), w1.std()

(tensor(-6.8854e-05), tensor(0.0504))

In [51]:
w1.shape

torch.Size([784, 50])

In [52]:
import torch.nn

In [54]:
torch.nn.Linear(m,nh)

Linear(in_features=784, out_features=50, bias=True)

In [55]:
torch.nn.Linear.forward??

Signature: torch.nn.Linear.forward(self, input)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, input):
        return F.linear(input, self.weight, self.bias)
File:      ~/anaconda3/envs/fastai-py37/lib/python3.7/site-packages/torch/nn/modules/linear.py
Type:      function


In [56]:
torch.nn.functional.linear??

Signature: torch.nn.functional.linear(input, weight, bias=None)
Source:   
def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

    Shape:

        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())
        if bias is not None:
            output += bias
        ret = output
    return ret
File:      ~/anaconda3/envs/fastai-py37/lib/python3.7/site-packages/torch/nn/functional.py
Type:      function


In [57]:
torch.nn.Conv2d??

Init signature:
torch.nn.Conv2d(
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
    bias=True,
    padding_mode='zeros',
)
Source:        
class Conv2d(_ConvNd):
    r"""Applies a 2D convolution over an input signal composed of several input
    planes.

    In the simplest case, the output value of the layer with input size
    :math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
    can be precisely described as:

    .. math::
        \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
        \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


    where :math:`\star` is the valid 2D `cross-correlation`_ operator,
    :math:`N` is a batch size, :math:`C` denotes a number of channels,
    :math:`H` is a height of input planes in pixels, and :math:`W` is
    width in pixels.

    * :attr:`stride` controls the st

In [58]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/envs/fastai-py37/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


In [59]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [61]:
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.2069), tensor(0.9797))

In [62]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [63]:
%timeit -n 10 _=model(x_valid)

8.49 ms ± 642 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

# LOSS Function

In [65]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [69]:
y_train,y_valid = y_train.float(),y_valid.float()

In [70]:
preds = model(x_train)

In [71]:
preds.shape

torch.Size([50000, 1])

In [72]:
mse(preds, y_train)

tensor(25.6259)

In [74]:
# Gradient and backward pass
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [75]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [76]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [77]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [78]:
forward_and_backward(x_train, y_train)

In [79]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [80]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [81]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)